# Reconstructing Artery Wall from Point Cloud Using Deep Learning

Xinling Yu, Zhiyuan Ren, Ian Wu, Zheng Xu

## 1. Introduction

In the virtual surgery simulation system, the reconstruction of a highly precise 3D medical model is an effective method to improve the user’s visual telepresence. However, the traditional point cloud generation method based on subdivision and filling is unsatisfactory due to its low accuracy and slow speed. To address this problem, we present a novel 3D point cloud reconstructing model based on level set method.

## 2. Theory and Methods

1. Implicitly defined manifolds
2. Signed distance function and Eikonal PDE
3. Implicit geometric regularization

### Implicitly defined manifolds

The implicit representation of a manifold takes the form
\begin{equation}
\mathcal{M}=\left\{\boldsymbol{x} \in \mathbb{R}^{n} \mid f(\boldsymbol{x})=0\right\},
\end{equation}
so that $\boldsymbol{x} \in \mathcal{M}$ is a point on the manifold implicitly defined by the function $f: \mathbb{R}^{n} \rightarrow \mathbb{R}$. There are three different cases for the level-set function $f$
\begin{align}
&f(\boldsymbol{x})>0,~\boldsymbol{x}~\text{is on the one side of the manifold}, \\
&f(\boldsymbol{x})<0,~\boldsymbol{x}~\text{is on the other side of the manifold}, \\
&f(\boldsymbol{x})=0,~\boldsymbol{x}~\text{is exactly on the manifold}.
\end{align}

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%%html
<center><img src='Circle.png', width=400, height=400>
<center><img src='Aneurysm.png', width=600, height=600>

### Signed distance function and Eikonal PDE

If $\Omega$ is a subset of a metric space, $X$, with metric, $d$, then the signed distance function, $f$, is defined by
$$
f(\boldsymbol{x})= \begin{cases}d(\boldsymbol{x}, \partial \Omega) & \text { if } \boldsymbol{x} \in \Omega \\ -d(\boldsymbol{x}, \partial \Omega) & \text { if } \boldsymbol{x} \in \Omega^{c}\end{cases}
$$
where $\partial \Omega$ denotes the boundary of $\Omega$. For any $\boldsymbol{x} \in X$,
$$
d(\boldsymbol{x}, \partial \Omega):=\inf _{\boldsymbol{y} \in \partial \Omega} d(\boldsymbol{x}, \boldsymbol{y})
$$
where inf denotes the infimum. From the above definition, we know that the signed distance function can be used to represent a manifold implicitly. The question is: How we find a appropriate signed distance function $f$?

If $\Omega$ is a subset of the Euclidean space $\mathbb{R}^{n}$ with piecewise smooth boundary, then the signed distance function is differentiable almost everywhere, and its gradient satisfies the Eikonal PDE
$$
\|\nabla f\|=1 \text {, } \\
f(\boldsymbol{x}) = 0~~\text{on}~~\partial \Omega.
$$
Given an input ponit cloud on the surface, now we can find a signed distance function that defines the manifold by solving the above Eikonal PDE.

### Implicit geometric regularization

To solve the Eikonal PDE using deep learning, we use a neural network $f(\boldsymbol{x};\theta)$ to approximate the PDE solution $f$, then the parameter $\theta$ can be learned by minimizing the following loss
$$
\mathcal{L}(\theta) = \sum_{i \in \partial \Omega} \left|f\left(x_{i} ; \theta\right)\right|^{2} + \lambda \mathbb{E}_{x} \left(\left\|\nabla_{x} f(x ; \theta)\right\|-1\right)^{2},
$$
here the first term encourages $f$ to vanish on the surface and the second term forces $f$ to satisfy the Eikonal PDE.

## 3. Results

1. Dataset and problem setup
2. Reconstruction results

### Dataset and problem setup

We use the 3D Intracranial Aneurysm Dataset to assess the performance of our deep learning model. Specifically, given an input surface point cloud, we aim to reconstruct the artery wall by solving the Eikonal PDE. 

In [51]:
%%html
<center><img src='IntrA.png', width=1000, height=1000>

### Reconstruction Visualization

In [34]:
from pygel3d import graph
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd

In [35]:
def graph(path):
    result = hmesh.load(path)
    return result

#### Visualization 1 - 100 Epochs - 1000 Sample Points

In [36]:
jd.display(graph("Data/3_truth.ply"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

In [37]:
%%html
<left><img src='2_pc.png', width=500, height=500>

In [38]:
jd.display(graph("Data/3_output.ply"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

#### Visualization 2 - 100 Epochs - 2000 Sample Points

In [39]:
jd.display(graph("Data/4_truth.obj"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

In [40]:
%%html
<left><img src='4_pc.png', width=500, height=500>

In [41]:
jd.display(graph("Data/4_output.ply"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

#### Visualization 3 - 1000 Epochs - 1000 Sample Points

In [42]:
jd.display(graph("Data/ground_truth_5.obj"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

In [52]:
%%html
<left><img src='5_pc.png', width=500, height=500>

In [45]:
jd.display(graph("Data/output_mesh_5.ply"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

## Visualization 4 (Complex Structure) - 300 Epochs

### Test Case 1 - 10,000 Sample Points / 7x512 MLP

In [46]:
%%html
<center><img src='Artery_1e4.png' style="transform: rotate(180deg); width=1000, height=1000">

### Test Case 2 - 100,000 Sample Points / 7x512 MLP

In [47]:
%%html
<center><img src='Artery_1e5.png' style="transform: rotate(180deg); width=1000, height=1000">

### Test Case 3 - 100,000 Sample Points / 9x1024 MLP

In [48]:
%%html
<center><img src='Artery_1e5_BigNN.png' style="transform: rotate(180deg); width=1000, height=1000">

### Test Case 3 3D Visualization

In [49]:
jd.display(graph("Data/1/ArteryObjAN1.obj"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

In [50]:
jd.display(graph("Data/1/Artery_mesh_example1.ply"))

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array(…

#### Artery reconstruction using 100,000 points and a 9 $\times$ 1024 MLP

## 4. Discussion

## 5. Reference

1. UCSB ECE 594N Geometric Machine Learning for Biomedical Imaging and Shape Analysis: https://github.com/bioshape-lab/ece594n.
2. Signed distance function from Wikipedia: https://en.wikipedia.org/wiki/Signed_distance_function.
3. Yang, Xi, et al. "IntrA: 3D intracranial aneurysm dataset for deep learning." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2020.
4. Gropp, Amos, et al. "Implicit geometric regularization for learning shapes." arXiv preprint arXiv:2002.10099 (2020).
